In [5]:
import plotly.express as px
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
import plotly.graph_objs as go
from assign_y import assign_y
from construct_grid import make_grid

In [22]:
y_data = pd.read_csv('../raw_data/berlin_y.csv', sep='\t')
eu_cities = gpd.read_file('../raw_data/eu_cities.json')
berlin_shape = eu_cities[eu_cities['city_name']=='Berlin']
berlin_grid = make_grid(berlin_shape,1000)
y_data_gdf = gpd.GeoDataFrame(y_data[['decimalLongitude','decimalLatitude']], geometry=gpd.points_from_xy(x=y_data.decimalLongitude,y=y_data.decimalLatitude), crs='epsg:4326')
y_data_gdf = y_data_gdf.to_crs(epsg=3035)
gridded_y = assign_y(berlin_grid,y_data_gdf)
gridded_y['num_points_inside'] = np.log(gridded_y['num_points_inside'])
gridded_y = gridded_y.to_crs(epsg=4326)
url = "https://github.com/eurostat/Nuts2json/raw/master/2021/4326/20M/nutsrg_3.json"
european_countries = gpd.read_file(url)
gridded_y_json = gridded_y.geometry.__geo_interface__
location_name = berlin_shape['city_name']
location_lat = berlin_shape['geometry'].centroid.y.values[0]
location_lon = berlin_shape['geometry'].centroid.x.values[0]
location_point = Point(location_lon, location_lat)
location_country = european_countries[european_countries.contains(location_point)].iloc[0]
location_data = pd.DataFrame({"Name": [location_name], "Latitude": [location_lat], "Longitude": [location_lon]})
location_data['Name'] = location_data['Name'].astype(str)


/var/folders/dy/8nkgqf653sggg31kzy33mjdr0000gn/T/ipykernel_99467/3862336917.py:1: DtypeWarning:

Columns (2,14,17,38,39,40,41,43,44,48) have mixed types. Specify dtype option on import or set low_memory=False.

/Users/zilikons/.pyenv/versions/3.10.6/envs/demeter/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning:

divide by zero encountered in log

/var/folders/dy/8nkgqf653sggg31kzy33mjdr0000gn/T/ipykernel_99467/3862336917.py:14: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/var/folders/dy/8nkgqf653sggg31kzy33mjdr0000gn/T/ipykernel_99467/3862336917.py:15: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [40]:
gridded_y.to_file('../raw_data/berlin_grid_y.json', driver='GeoJSONSeq')

In [43]:
fig = px.choropleth_mapbox(location_data, geojson=location_country.geometry.__geo_interface__, color="Name",
locations='Name', featureidkey="properties.NUTS_ID",
mapbox_style="carto-positron", zoom=10, center={"lat": location_lat, "lon": location_lon},
opacity=0.8, labels={'Name': 'Location'}, hover_data=["Name"])
trace = go.Choroplethmapbox(
    geojson=gridded_y_json,
    locations=gridded_y.index,
    z=gridded_y['num_points_inside'],
    colorscale='Reds',
    marker_opacity=0.5,
    marker_line_width=0,
    name='BioScore'
)

fig.add_trace(trace)
fig.update_layout(
    showlegend = False
)

fig.show()

ValueError: 
    Invalid element(s) received for the 'hoverinfo' property of choroplethmapbox
        Invalid elements include: ['num_points_inside']

    The 'hoverinfo' property is a flaglist and may be specified
    as a string containing:
      - Any combination of ['location', 'z', 'text', 'name'] joined with '+' characters
        (e.g. 'location+z')
        OR exactly one of ['all', 'none', 'skip'] (e.g. 'skip')
      - A list or array of the above